In [1]:
import random
import json
import pickle
import numpy as np
import pandas as pd
import tensorflow 
import keras
import nltk 
#nltk.download()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, Activation, GlobalMaxPooling1D, Dropout, Bidirectional, LSTM
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
intents = json.loads(open('intents.json').read())

In [4]:
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
words = [word for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

### **Calculate X**

In [6]:
question = []
for doc in intents['intents']:
  question.append(doc['patterns'])

In [7]:
question_list = []
arr = [[question_list.append(x) for x in q] for q in question]

In [8]:
question_list

['Can a student who has received a temporary certificate of completion be able to return to education?',
 'Can a graduate student who has a temporary certificate return to study?',
 'Can a graduate associate who has a temporary certificate return to study?',
 "What is the email address of the university's educational administration?",
 "What is the university's educational management email?",
 'Educational management email',
 'University educational management email',
 'How can scholarship graduates of the Ministry of Science receive a temporary certificate?',
 'How to get the temporary certificate of scholarship graduates of the Ministry of Science',
 'How can graduates of the scholarship of the Ministry of Science receive a temporary certificate?',
 'What is in the temporary certificate of completion of education?',
 'Obligations of temporary certificate of completion of education',
 'What are the obligations included in the temporary certificate of graduation?',
 'What are the oblig

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(question_list)

In [10]:
with open('tokenizer.h5', 'wb') as f:
    pickle.dump(tokenizer, f)

In [11]:
vocab_size = len(tokenizer.word_index)+1

In [12]:
vocab_size

541

In [13]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [14]:
data_to_sequences = tokenizer.texts_to_sequences(question_list)
maxlen = max_length(data_to_sequences)

In [15]:
maxlen

40

In [16]:
pad_data_to_sequences = pad_sequences(data_to_sequences, maxlen=maxlen)

In [17]:
pad_data_to_sequences

array([[  0,   0,   0, ...,  92,   3,   7],
       [  0,   0,   0, ...,  92,   3,  40],
       [  0,   0,   0, ...,  92,   3,  40],
       ...,
       [  0,   0,   0, ...,   1, 305,  63],
       [  0,   0,   0, ...,  81,  91, 412],
       [  0,   0,   0, ...,   1, 124, 414]], dtype=int32)

In [18]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [19]:
random.shuffle(training)
training = np.array(training)

<ipython-input-19-cc0bc1dae1db>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


### **Calculate Y**

In [20]:
tag = []
for doc in documents:
  tag.append(doc[1])

In [21]:
Y = pd.get_dummies(tag)

In [22]:
num_unique_tags = len(Y.nunique())

In [23]:
num_unique_tags

127

### **X and Y**

In [24]:
train_x = pad_data_to_sequences
train_y = np.array(Y)

### **Network**

In [25]:
embedding_dim = 50

In [26]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(Bidirectional(LSTM(200, return_sequences=True)))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_unique_tags, activation='softmax'))

In [27]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 50)            27050     
                                                                 
 bidirectional (Bidirectiona  (None, 40, 400)          401600    
 l)                                                              
                                                                 
 global_max_pooling1d (Globa  (None, 400)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 400)               0         
                                                                 
 dense (Dense)               (None, 200)               80200     
                                                                 
 dropout_1 (Dropout)         (None, 200)               0

In [29]:
hist = model.fit(train_x, train_y, epochs=200, batch_size=128, verbose=1)

Epoch 1/200
3/3 [==============================] - 6s 403ms/step - loss: 4.8444 - accuracy: 0.0134
Epoch 2/200
3/3 [==============================] - 1s 384ms/step - loss: 4.8395 - accuracy: 0.0027
Epoch 3/200
3/3 [==============================] - 1s 374ms/step - loss: 4.8365 - accuracy: 0.0214
Epoch 4/200
3/3 [==============================] - 2s 645ms/step - loss: 4.8347 - accuracy: 0.0054
Epoch 5/200
3/3 [==============================] - 2s 657ms/step - loss: 4.8306 - accuracy: 0.0188
Epoch 6/200
3/3 [==============================] - 1s 420ms/step - loss: 4.8206 - accuracy: 0.0214
Epoch 7/200
3/3 [==============================] - 1s 400ms/step - loss: 4.8112 - accuracy: 0.0214
Epoch 8/200
3/3 [==============================] - 1s 391ms/step - loss: 4.8036 - accuracy: 0.0107
Epoch 9/200
3/3 [==============================] - 1s 535ms/step - loss: 4.8055 - accuracy: 0.0134
Epoch 10/200
3/3 [==============================] - 2s 574ms/step - loss: 4.7849 - accuracy: 0.0214
Epoch 11/

In [ ]:
# weighs = model.get_weights();
# pickle.dump(weighs, open('model.pkl', 'wb'))

In [30]:
model.save('model.h5', hist)

## **Chatbot**

In [31]:
intents = json.loads(open('intents.json').read())

In [32]:
# load model
model = load_model('model.h5')

In [33]:
#loaded_model = pickle.load(open("model.pkl", 'rb'))

In [34]:
# load tokenizer
with open('tokenizer.h5', 'rb') as f:
    tokenizer = pickle.load(f)

In [35]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    return sentence_words

In [38]:
MAX_LENGTH = 40
def prediction(text):
  text_to_sequences = tokenizer.texts_to_sequences([text])
  pad_text_to_sequences = pad_sequences(text_to_sequences, maxlen=MAX_LENGTH)
  tag = model.predict(pad_text_to_sequences)[0]
  tag_index = np.where(tag == max(tag))[0][0]
  result = Y.columns[tag_index]
  return result

In [44]:
prediction('Can a student who has received a temporary certificate of completion be able to return to education')

1/1 [==============================] - 0s 104ms/step


'Can a student who has received a temporary certificate of completion be able to return to education?'

In [42]:
def get_response(text):
  tag = prediction(text)
  for data in intents['intents']:
    if data['tag'] == tag:
      result = random.choice(data['response'])
      break
  return result    

In [45]:
get_response('Can a student who has received a temporary certificate of completion be able to return to education')

1/1 [==============================] - 0s 51ms/step


'If he receives a temporary certificate, he cannot return to study.'

In [ ]:
while True:
	message = input("Enter a message:")
	res = get_response(message)
	print(res)

Enter a message:Can a student who has received a temporary certificate of completion be able to return to education
1/1 [==============================] - 0s 32ms/step
If he receives a temporary certificate, he cannot return to study.
Enter a message:What is in the temporary certificate of completion of education
1/1 [==============================] - 0s 29ms/step
Due to the free education in the above-mentioned study level, daily students have the obligation to serve for free education based on the admission of the entrance exam area, and all obligations must be settled when receiving the educational encyclopedia. (Complete description of settlement of obligations in the receiving section of the encyclopedia)
